In [89]:
import nfl_data_py as nfl
import pandas as pd

# Exploring PBP and Weekly NFL data

## GOAL: create features to accurately predict fantasy points given player, year, week

- Position
- AVG team points
- AVG team points over last 3 weeks
- Last week's team total
- AVG stats per week (passing, rushing, recieving, td, fantasy pts, snap %, targets (RZ?), carries)
- AVG stats over last n weeks (same) (TUNE n??)
- Last weeks stats (same)
- AVG OPP DEF stats per week (pass yds, rush yds, pass/run tds, pts allowed)
- AVG OPP DEF stats over last n weeks (same) (TUNE n??)
- Last week's OPP DEF stats (same)
- Vegas (over/under, applied pts)

## Example player: Ceedee Lamb Week 10, 2023
### Step 1: AVG stats per week


In [90]:
# load in weekly data
weekly = nfl.import_weekly_data([2022,2023,2024])

Downcasting floats.


In [91]:
weekly.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [92]:
# add columns
weekly['fumbles'] = weekly['rushing_fumbles'] + weekly['receiving_fumbles'] + weekly['passing_first_downs']
weekly['fumbles_lost'] = weekly['rushing_fumbles_lost'] + weekly['receiving_fumbles_lost'] + weekly['passing_first_downs']
weekly['first_downs'] = weekly['passing_first_downs'] + weekly['rushing_first_downs'] + weekly['receiving_first_downs']


# get data for ceedee lamb in 2023 before wk 10
cd = weekly[(weekly['player_display_name'] == 'CeeDee Lamb') & (weekly['season'] == 2023) & (weekly['week'] < 10)]

# name and position
player_name = cd['player_display_name'].iloc[0]
position = cd['position'].iloc[0]
team = cd['recent_team'].iloc[0]

# average ceedees data across columns
cols_to_avg = ['passing_yards', 'passing_tds', 'interceptions', 'passing_epa',
            'carries', 'rushing_yards', 'rushing_tds', 'fumbles',
            'fumbles_lost', 'rushing_epa','receptions', 'targets',
            'receiving_yards', 'receiving_tds','receiving_air_yards','receiving_yards_after_catch',
            'receiving_epa', 'target_share', 'air_yards_share', 'fantasy_points_ppr',
            'first_downs']
cd_avg = cd[cols_to_avg].mean()

cd_avg

passing_yards                    0.000000
passing_tds                      0.000000
interceptions                    0.000000
passing_epa                           NaN
carries                          0.500000
rushing_yards                    4.125000
rushing_tds                      0.000000
fumbles                          0.250000
fumbles_lost                     0.125000
rushing_epa                      0.499025
receptions                       7.125000
targets                          9.000000
receiving_yards                103.000000
receiving_tds                    0.375000
receiving_air_yards             93.250000
receiving_yards_after_catch     37.125000
receiving_epa                    6.266804
target_share                     0.258531
air_yards_share                  0.348478
fantasy_points_ppr              19.837500
first_downs                      4.750000
dtype: float64

### Step 2: AVG stats over last 3 weeks

In [93]:
cd_last_3 = cd.tail(3)

cd_recent_avg = cd_last_3[cols_to_avg].mean()

cd_recent_avg

passing_yards                    0.000000
passing_tds                      0.000000
interceptions                    0.000000
passing_epa                           NaN
carries                          0.333333
rushing_yards                    4.000000
rushing_tds                      0.000000
fumbles                          0.333333
fumbles_lost                     0.333333
rushing_epa                      0.791323
receptions                      10.000000
targets                         12.333333
receiving_yards                155.333328
receiving_tds                    0.666667
receiving_air_yards            154.666672
receiving_yards_after_catch     50.666668
receiving_epa                   10.662650
target_share                     0.336245
air_yards_share                  0.457713
fantasy_points_ppr              29.266668
first_downs                      7.333333
dtype: float64

### Step 3: Stats from last week

In [94]:
cd_last_1 = cd[cols_to_avg].tail(1)

cd_last_1

,passing_yards,passing_tds,interceptions,passing_epa,carries,rushing_yards,rushing_tds,fumbles,fumbles_lost,rushing_epa,...,targets,receiving_yards,receiving_tds,receiving_air_yards,receiving_yards_after_catch,receiving_epa,target_share,air_yards_share,fantasy_points_ppr,first_downs
3098,0.0,0,0.0,NaN,0,0.0,0,1.0,1.0,NaN,...,16,191.0,0,234.0,57.0,8.701958,0.363636,0.557143,28.1,7.0


Now, merge name, pos, team with averages, last 3, and last 1

In [95]:
# Add identifiers
cd_avg.index = [f'{col}_avg' for col in cd_avg.index]
cd_recent_avg.index = [f'{col}_last3' for col in cd_recent_avg.index]
cd_last_1 = cd_last_1.rename(lambda col: f'{col}_last1', axis=1).iloc[0]

# Add name and position
id_info = pd.Series({'player_display_name': player_name,
                      'position': position,
                      'team': team,
                      'week': 10,
                      'season': 2023})

# Combine all pieces into one Series, then make a DataFrame
row = pd.concat([id_info, cd_avg, cd_recent_avg, cd_last_1])
final_row_df = row.to_frame().T  # to_frame().T turns Series into a one-row DataFrame

final_row_df.head()  # optional: preview

,player_display_name,position,team,week,season,passing_yards_avg,passing_tds_avg,interceptions_avg,passing_epa_avg,carries_avg,...,targets_last1,receiving_yards_last1,receiving_tds_last1,receiving_air_yards_last1,receiving_yards_after_catch_last1,receiving_epa_last1,target_share_last1,air_yards_share_last1,fantasy_points_ppr_last1,first_downs_last1
0,CeeDee Lamb,WR,DAL,10,2023,0.0,0.0,0.0,NaN,0.5,...,16.0,191.0,0.0,234.0,57.0,8.701958,0.363636,0.557143,28.1,7.0


### Step 4: AVG Team Offensive Totals (season long, last 3, last 1)

In [96]:
# import play by play data
pbp = nfl.import_pbp_data([2022,2023,2024])

2022 done.
2023 done.
2024 done.
Downcasting floats.


In [97]:
# get only weeks where fantasy football is played
pbp = pbp[(pbp['week'].between(1, 17)) & (pbp['season_type'] == 'REG')]

In [ ]:
# aggregate by week, team
# this will get us both offensive and defensive stats by week
pbp = pbp[(pbp['week'].between(1, 17)) & (pbp['season_type'] == 'REG')]
pbp['turnover'] = pbp['interception'] + pbp['fumble_lost']
weekly_stats = (
    pbp.groupby(['game_id', 'posteam', 'defteam', 'season', 'week'])
    .agg(team_passing_yards = ('passing_yards', 'sum'),
         team_rushing_yards = ('rushing_yards', 'sum'),
         team_passing_tds = ('pass_touchdown', 'sum'),
         team_rushing_tds = ('rush_touchdown', 'sum'),
         team_total_plays = ('posteam', 'count'),
         team_first_downs = ('first_down', 'sum'),
         team_yards_per_play = ('yards_gained', 'mean'),
         team_turnovers = ('turnover', 'sum'))
    ).reset_index()

weekly_stats

,game_id,posteam,defteam,season,week,team_passing_yards,team_rushing_yards,team_passing_tds,team_rushing_tds,team_total_plays,team_first_downs,team_yards_per_play,team_turnovers
0,2022_01_BAL_NYJ,BAL,NYJ,2022,1,213.0,63.0,3.0,0.0,71,13.0,3.971014,1.0
1,2022_01_BAL_NYJ,NYJ,BAL,2022,1,309.0,83.0,1.0,0.0,99,24.0,3.877551,2.0
2,2022_01_BUF_LA,BUF,LA,2022,1,297.0,121.0,3.0,1.0,70,23.0,6.073529,4.0
3,2022_01_BUF_LA,LA,BUF,2022,1,240.0,52.0,1.0,0.0,82,19.0,3.000000,3.0
4,2022_01_CLE_CAR,CAR,CLE,2022,1,235.0,54.0,1.0,2.0,74,14.0,3.676056,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2024_17_NYJ_BUF,NYJ,BUF,2024,17,195.0,112.0,2.0,0.0,76,20.0,3.773333,3.0
1530,2024_17_SEA_CHI,CHI,SEA,2024,17,122.0,103.0,0.0,0.0,74,11.0,2.452055,1.0
1531,2024_17_SEA_CHI,SEA,CHI,2024,17,160.0,122.0,0.0,0.0,65,12.0,4.206349,1.0
1532,2024_17_TEN_JAX,JAX,TEN,2024,17,174.0,122.0,2.0,0.0,72,18.0,4.214286,0.0


In [99]:
# save for later
weekly_stats.to_csv('/Users/nicholaspatrick/Desktop/projects/Fantasy-Football-Predictor-2025/data/team_weekly_stats.csv', index=True)

Now, calculate data for DAL offense 2023 wk 1-9

In [100]:
DAL_off = weekly_stats[(weekly_stats['posteam'] == 'DAL') & (weekly_stats['season'] == 2023) & (weekly_stats['week'] < 10)]

DAL_off

,game_id,posteam,defteam,season,week,team_passing_yards,team_rushing_yards,team_passing_tds,team_rushing_tds,team_total_plays,team_first_downs,team_yards_per_play,team_turnovers
518,2023_01_DAL_NYG,DAL,NYG,2023,1,143.0,122.0,0.0,3.0,72,18.0,3.785714,0.0
566,2023_02_NYJ_DAL,DAL,NYJ,2023,2,255.0,134.0,2.0,0.0,104,26.0,3.801980,0.0
583,2023_03_DAL_ARI,DAL,ARI,2023,3,249.0,185.0,1.0,0.0,96,25.0,4.378947,1.0
628,2023_04_NE_DAL,DAL,NE,2023,4,281.0,124.0,1.0,1.0,86,22.0,4.511905,0.0
646,2023_05_DAL_SF,DAL,SF,2023,5,161.0,57.0,1.0,0.0,63,8.0,3.126984,4.0
672,2023_06_DAL_LAC,DAL,LAC,2023,6,272.0,96.0,1.0,1.0,80,20.0,4.441558,0.0
738,2023_08_LA_DAL,DAL,LA,2023,8,307.0,102.0,4.0,0.0,82,24.0,4.777778,1.0
760,2023_09_DAL_PHI,DAL,PHI,2023,9,374.0,73.0,3.0,0.0,90,25.0,4.613636,1.0


Season averages

In [101]:
DAL_off_avg = (
    DAL_off.groupby('posteam').
    agg(team_passing_yards_avg = ('team_passing_yards', 'mean'),
         team_rushing_yards_avg = ('team_rushing_yards', 'mean'),
         team_passing_tds_avg = ('team_passing_tds', 'mean'),
         team_rushing_tds_avg = ('team_rushing_tds', 'mean'),
         team_total_plays_avg = ('team_total_plays', 'mean'),
         team_first_downs_avg = ('team_first_downs', 'mean'),
         team_yards_per_play_avg = ('team_yards_per_play', 'mean'),
         team_turnovers_avg = ('team_turnovers', 'mean'))
    ).reset_index()

DAL_off_avg

,posteam,team_passing_yards_avg,team_rushing_yards_avg,team_passing_tds_avg,team_rushing_tds_avg,team_total_plays_avg,team_first_downs_avg,team_yards_per_play_avg,team_turnovers_avg
0,DAL,255.25,111.625,1.625,0.625,84.125,21.0,4.179813,0.875


Last 3 week average

In [102]:
DAL_off_last3 = (
    DAL_off.tail(3).groupby('posteam').
    agg(team_passing_yards_last3 = ('team_passing_yards', 'mean'),
         team_rushing_yards_last3 = ('team_rushing_yards', 'mean'),
         team_passing_tds_last3 = ('team_passing_tds', 'mean'),
         team_rushing_tds_last3 = ('team_rushing_tds', 'mean'),
         team_total_plays_last3 = ('team_total_plays', 'mean'),
         team_first_downs_last3 = ('team_first_downs', 'mean'),
         team_yards_per_play_last3 = ('team_yards_per_play', 'mean'),
         team_turnovers_last3 = ('team_turnovers', 'mean'))
    ).reset_index() 

DAL_off_last3

,posteam,team_passing_yards_last3,team_rushing_yards_last3,team_passing_tds_last3,team_rushing_tds_last3,team_total_plays_last3,team_first_downs_last3,team_yards_per_play_last3,team_turnovers_last3
0,DAL,317.666656,90.333336,2.666667,0.333333,84.0,23.0,4.610991,0.666667


Last week's stats

In [103]:
DAL_off_last1 = DAL_off.tail(1).rename(lambda col: f'{col}_last1', axis=1).drop(['game_id_last1',
                                                                                 'defteam_last1',
                                                                                 'season_last1',
                                                                                 'week_last1'], axis = 1).rename({'posteam_last1': 'posteam'}, axis = 1)

DAL_off_last1

,posteam,team_passing_yards_last1,team_rushing_yards_last1,team_passing_tds_last1,team_rushing_tds_last1,team_total_plays_last1,team_first_downs_last1,team_yards_per_play_last1,team_turnovers_last1
760,DAL,374.0,73.0,3.0,0.0,90,25.0,4.613636,1.0


Finally, combine with final dataset (row)

In [104]:
final_row_df = (final_row_df.
                merge(DAL_off_avg,
                      left_on=['team'],
                      right_on=['posteam']).
                      merge(DAL_off_last3,
                      left_on=['team'],
                      right_on=['posteam']).
                      merge(DAL_off_last1,
                      left_on=['team'],
                      right_on=['posteam'])
)
                      

final_row_df

,player_display_name,position,team,week,season,passing_yards_avg,passing_tds_avg,interceptions_avg,passing_epa_avg,carries_avg,...,team_turnovers_last3,posteam,team_passing_yards_last1,team_rushing_yards_last1,team_passing_tds_last1,team_rushing_tds_last1,team_total_plays_last1,team_first_downs_last1,team_yards_per_play_last1,team_turnovers_last1
0,CeeDee Lamb,WR,DAL,10,2023,0.0,0.0,0.0,NaN,0.5,...,0.666667,DAL,374.0,73.0,3.0,0.0,90,25.0,4.613636,1.0


### Step 5: AVG Opponent stats (season, last 3, last 1)

In [105]:
# find opponent

schedules = nfl.import_schedules([2022,2023,2024])

game = schedules[
    (schedules['season'] == 2023) & (schedules['week'] == 10) &
    ((schedules['home_team'] == 'DAL') | (schedules['away_team'] == 'DAL'))]

if game['home_team'].iloc[0] == 'DAL':
    opponent = game['away_team'].iloc[0]
else:
    opponent = game['home_team'].iloc[0]

opponent

'NYG'

In [106]:
# get stats of opponents defense
opponent_weekly = weekly_stats[(weekly_stats['defteam'] == opponent) & (weekly_stats['season'] == 2023) & (weekly_stats['week'] < 10)]

opponent_weekly

,game_id,posteam,defteam,season,week,team_passing_yards,team_rushing_yards,team_passing_tds,team_rushing_tds,team_total_plays,team_first_downs,team_yards_per_play,team_turnovers
518,2023_01_DAL_NYG,DAL,NYG,2023,1,143.0,122.0,0.0,3.0,72,18.0,3.785714,0.0
564,2023_02_NYG_ARI,ARI,NYG,2023,2,228.0,151.0,1.0,2.0,87,23.0,4.535714,0.0
599,2023_03_NYG_SF,SF,NYG,2023,3,310.0,141.0,2.0,1.0,98,26.0,4.642105,0.0
633,2023_04_SEA_NYG,SEA,NYG,2023,4,173.0,121.0,1.0,1.0,69,13.0,4.132353,0.0
658,2023_05_NYG_MIA,MIA,NYG,2023,5,308.0,222.0,2.0,2.0,68,22.0,7.820896,3.0
686,2023_06_NYG_BUF,BUF,NYG,2023,6,169.0,128.0,2.0,0.0,75,22.0,4.013514,2.0
721,2023_07_WAS_NYG,WAS,NYG,2023,7,249.0,76.0,0.0,1.0,91,13.0,3.067416,2.0
749,2023_08_NYJ_NYG,NYJ,NYG,2023,8,240.0,58.0,1.0,0.0,84,12.0,3.024096,2.0
772,2023_09_NYG_LV,LV,NYG,2023,9,209.0,125.0,0.0,3.0,75,15.0,4.453333,0.0


In [107]:
# average opponent defensive stats

opp_def_avg = (
    opponent_weekly.groupby('defteam').
    agg(opp_passing_yards_avg = ('team_passing_yards', 'mean'),
         opp_rushing_yards_avg = ('team_rushing_yards', 'mean'),
         opp_passing_tds_avg = ('team_passing_tds', 'mean'),
         opp_rushing_tds_avg = ('team_rushing_tds', 'mean'),
         opp_total_plays_avg = ('team_total_plays', 'mean'),
         opp_first_downs_avg = ('team_first_downs', 'mean'),
         opp_yards_per_play_avg = ('team_yards_per_play', 'mean'),
         opp_turnovers_avg = ('team_turnovers', 'mean'))
    ).reset_index()

opp_def_avg

,defteam,opp_passing_yards_avg,opp_rushing_yards_avg,opp_passing_tds_avg,opp_rushing_tds_avg,opp_total_plays_avg,opp_first_downs_avg,opp_yards_per_play_avg,opp_turnovers_avg
0,NYG,225.444443,127.111115,1.0,1.444444,79.888889,18.222221,4.386127,1.0


In [108]:
# average opponent defense over last 3 weeks
opp_def_last3 = (
    opponent_weekly.tail(3).groupby('defteam').
    agg(opp_passing_yards_last3 = ('team_passing_yards', 'mean'),
         opp_rushing_yards_last3= ('team_rushing_yards', 'mean'),
         opp_passing_tds_last3 = ('team_passing_tds', 'mean'),
         opp_rushing_tds_last3 = ('team_rushing_tds', 'mean'),
         opp_total_plays_last3 = ('team_total_plays', 'mean'),
         opp_first_downs_last3 = ('team_first_downs', 'mean'),
         opp_yards_per_play_last3 = ('team_yards_per_play', 'mean'),
         opp_turnovers_last3 = ('team_turnovers', 'mean'))
    ).reset_index()

opp_def_last3

,defteam,opp_passing_yards_last3,opp_rushing_yards_last3,opp_passing_tds_last3,opp_rushing_tds_last3,opp_total_plays_last3,opp_first_downs_last3,opp_yards_per_play_last3,opp_turnovers_last3
0,NYG,232.666672,86.333336,0.333333,1.333333,83.333333,13.333333,3.514949,1.333333


In [109]:
# opponent defense last week

opp_def_last1 = opponent_weekly.tail(1).rename(lambda col: f'{col}_last1', axis=1).drop(['game_id_last1',
                                                                                 'posteam_last1',
                                                                                 'season_last1',
                                                                                 'week_last1'], axis = 1).rename({'defteam_last1': 'defteam'}, axis = 1)

opp_def_last1 = opp_def_last1.rename(columns={col: col.replace('team_', 'opp_') for col in opp_def_last1.columns if col.startswith('team_')})
opp_def_last1

,defteam,opp_passing_yards_last1,opp_rushing_yards_last1,opp_passing_tds_last1,opp_rushing_tds_last1,opp_total_plays_last1,opp_first_downs_last1,opp_yards_per_play_last1,opp_turnovers_last1
772,NYG,209.0,125.0,0.0,3.0,75,15.0,4.453333,0.0


In [110]:
# merge all opponent stats and add to final row

opp_def = opp_def_avg.merge(opp_def_last1, on='defteam').merge(opp_def_last3, on='defteam').drop('defteam', axis = 1)

opp_def


# merge with full row

final_row_df = pd.concat([final_row_df, opp_def], axis=1)
final_row_df

,player_display_name,position,team,week,season,passing_yards_avg,passing_tds_avg,interceptions_avg,passing_epa_avg,carries_avg,...,opp_yards_per_play_last1,opp_turnovers_last1,opp_passing_yards_last3,opp_rushing_yards_last3,opp_passing_tds_last3,opp_rushing_tds_last3,opp_total_plays_last3,opp_first_downs_last3,opp_yards_per_play_last3,opp_turnovers_last3
0,CeeDee Lamb,WR,DAL,10,2023,0.0,0.0,0.0,NaN,0.5,...,4.453333,0.0,232.666672,86.333336,0.333333,1.333333,83.333333,13.333333,3.514949,1.333333


### Step 6: Team & Opponent Point Totals

In [111]:
team = 'DAL'
season = 2023
week = 10

# Team point totals
team_games = schedules[
    ((schedules['home_team'] == team) | (schedules['away_team'] == team)) &
    (schedules['season'] == season) &
    (schedules['week'] < week)
]


team_games['team_points'] = team_games.apply(
    lambda row: row['home_score'] if row['home_team'] == team else row['away_score'], axis=1
)

# add to final row
final_row_df['team_points_avg'] = team_games['team_points'].mean()
final_row_df['team_points_last1'] = team_games['team_points'].iloc[-1]
final_row_df['team_points_last3'] = team_games['team_points'].tail(3).mean()

# Opponent DEF point totals
opp_games = schedules[
    ((schedules['home_team'] == opponent) | (schedules['away_team'] == opponent)) &
    (schedules['season'] == season) &
    (schedules['week'] < week)
].copy()

# points allowed
opp_games['points_allowed'] = opp_games.apply(
    lambda row: row['away_score'] if row['home_team'] == opponent else row['home_score'], axis=1
)

# add to final row 
final_row_df['opp_pts_allowed_avg'] = opp_games['points_allowed'].mean()
final_row_df['opp_pts_allowed_last1'] = opp_games['points_allowed'].iloc[-1]
final_row_df['opp_pts_allowed_last3'] = opp_games['points_allowed'].tail(3).mean()


final_row_df = final_row_df.drop(columns=final_row_df.filter(regex="^posteam").columns)



<positron-console-cell-111>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Step 7: Vegas Stats

In [112]:
vegas_stats = schedules[
    ((schedules['home_team'] == team) | (schedules['away_team'] == team)) &
    (schedules['season'] == season) &
    (schedules['week'] == week)
]


if team == vegas_stats['home_team'].iloc[0]:
    vegas_stats['is_home'] = 1
else:
    vegas_stats['is_home'] = 0

vegas_stats = vegas_stats[['is_home', 'spread_line', 'total_line']]


vegas_stats['implied_team_total'] = (vegas_stats['total_line'] + vegas_stats['spread_line'] * (2 * vegas_stats['is_home'] - 1)) / 2

vegas_stats.reset_index(inplace=True)

# add to final row
final_row_df = pd.concat([final_row_df, vegas_stats], axis=1, )
final_row_df

<positron-console-cell-112>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,player_display_name,position,team,week,season,passing_yards_avg,passing_tds_avg,interceptions_avg,passing_epa_avg,carries_avg,...,team_points_last1,team_points_last3,opp_pts_allowed_avg,opp_pts_allowed_last1,opp_pts_allowed_last3,index,is_home,spread_line,total_line,implied_team_total
0,CeeDee Lamb,WR,DAL,10,2023,0.0,0.0,0.0,NaN,0.5,...,23.0,28.666667,24.111111,30.0,16.666667,6567,1,17.5,38.5,28.0


## Fixing Problem for weeks 1-3

- include last seasons data for averages
- use last 3 games going back to last year

### Example: CeeDee Lamb Week 1 2023


player data

In [141]:

player_name = 'CeeDee Lamb'
season = 2023
week = 3

player = weekly[
            (weekly['player_display_name'] == player_name) & 
            ((weekly['season'] == season) | (weekly['season'] == (season - 1)))]

player = player[~((player['season'] == season) & (player['week'] >= week))]


player

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,fumbles,fumbles_lost,first_downs
3957,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,1,REG,...,0.268519,0.268293,0.297521,0.610703,0.0,2.9,4.900000,0.0,0.0,2.0
3958,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,2,REG,...,0.750000,0.366667,0.561798,0.943258,0.0,8.1,15.100000,0.0,0.0,5.0
3959,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,3,REG,...,0.669231,0.400000,0.511811,0.958268,0.0,14.7,22.700001,0.0,0.0,6.0
3960,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,4,REG,...,0.950980,0.307692,0.453333,0.778872,0.0,15.7,21.700001,0.0,0.0,4.0
3961,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,5,REG,...,0.868852,0.500000,0.521368,1.114957,0.0,5.3,10.300000,0.0,0.0,2.0
3962,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,6,REG,...,1.046154,0.270270,0.220339,0.559643,0.0,7.7,12.700000,0.0,0.0,3.0
3963,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,7,REG,...,0.660377,0.240000,0.392593,0.634815,0.0,6.6,10.600000,0.0,0.0,4.0
3964,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,8,REG,...,0.855556,0.269231,0.463918,0.728588,0.0,13.8,18.799999,0.0,0.0,4.0
3965,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,10,REG,...,1.013514,0.340909,0.472843,0.842354,0.0,27.0,38.000000,0.0,0.0,8.0
3966,00-0036358,C.Lamb,CeeDee Lamb,WR,WR,https://static.www.nfl.com/image/private/f_aut...,DAL,2022,11,REG,...,1.730769,0.166667,0.189781,0.382847,0.0,4.5,9.500000,0.0,0.0,2.0


team data

In [144]:
# stats
team_off = weekly_stats[
            (weekly_stats['posteam'] == team) & 
            ((weekly_stats['season'] == season) | (weekly_stats['season'] == (season - 1)))]

            

team_off = team_off[~((team_off['season'] == season) & (team_off['week'] >= week))]


print(team_off)

# points

team_games = schedules[
    ((schedules['home_team'] == team) | (schedules['away_team'] == team)) &
    ((schedules['season'] == season) | (schedules['season'] == (season -1)))]
    
team_games = team_games[~((team_games['season'] == season) & (team_games['week'] >= week))]

team_games

             game_id posteam defteam  season  week  team_passing_yards  team_rushing_yards  team_passing_tds  team_rushing_tds  \
30    2022_01_TB_DAL     DAL      TB    2022     1               198.0                71.0               0.0               0.0   
41   2022_02_CIN_DAL     DAL     CIN    2022     2               235.0               107.0               1.0               1.0   
72   2022_03_DAL_NYG     DAL     NYG    2022     3               215.0               176.0               1.0               1.0   
126  2022_04_WAS_DAL     DAL     WAS    2022     4               223.0                62.0               2.0               0.0   
134   2022_05_DAL_LA     DAL      LA    2022     5               102.0               163.0               0.0               1.0   
170  2022_06_DAL_PHI     DAL     PHI    2022     6               181.0               134.0               1.0               1.0   
194  2022_07_DET_DAL     DAL     DET    2022     7               207.0               139.0

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6151,2022_01_TB_DAL,2022,REG,1,2022-09-11,Sunday,20:20,TB,19.0,DAL,...,NaN,00-0019596,00-0033077,Tom Brady,Dak Prescott,Todd Bowles,Mike McCarthy,Ron Torbert,DAL00,AT&T Stadium
6163,2022_02_CIN_DAL,2022,REG,2,2022-09-18,Sunday,16:25,CIN,17.0,DAL,...,NaN,00-0036442,00-0033662,Joe Burrow,Cooper Rush,Zac Taylor,Mike McCarthy,Brad Allen,DAL00,AT&T Stadium
6184,2022_03_DAL_NYG,2022,REG,3,2022-09-26,Monday,20:15,DAL,23.0,NYG,...,NaN,00-0033662,00-0035710,Cooper Rush,Daniel Jones,Mike McCarthy,Brian Daboll,Land Clark,NYC01,MetLife Stadium
6189,2022_04_WAS_DAL,2022,REG,4,2022-10-02,Sunday,13:00,WAS,10.0,DAL,...,NaN,00-0032950,00-0033662,Carson Wentz,Cooper Rush,Ron Rivera,Mike McCarthy,Shawn Smith,DAL00,AT&T Stadium
6214,2022_05_DAL_LA,2022,REG,5,2022-10-09,Sunday,16:25,DAL,22.0,LA,...,NaN,00-0033662,00-0026498,Cooper Rush,Matthew Stafford,Mike McCarthy,Sean McVay,Bill Vinovich,LAX01,SoFi Stadium
6229,2022_06_DAL_PHI,2022,REG,6,2022-10-16,Sunday,20:20,DAL,17.0,PHI,...,NaN,00-0033662,00-0036389,Cooper Rush,Jalen Hurts,Mike McCarthy,Nick Sirianni,John Hussey,PHI00,Lincoln Financial Field
6235,2022_07_DET_DAL,2022,REG,7,2022-10-23,Sunday,13:00,DET,6.0,DAL,...,NaN,00-0033106,00-0033077,Jared Goff,Dak Prescott,Dan Campbell,Mike McCarthy,Adrian Hill,DAL00,AT&T Stadium
6248,2022_08_CHI_DAL,2022,REG,8,2022-10-30,Sunday,13:00,CHI,29.0,DAL,...,NaN,00-0036945,00-0033077,Justin Fields,Dak Prescott,Matt Eberflus,Mike McCarthy,Brad Rogers,DAL00,AT&T Stadium
6283,2022_10_DAL_GB,2022,REG,10,2022-11-13,Sunday,16:25,DAL,28.0,GB,...,8.0,00-0033077,00-0023459,Dak Prescott,Aaron Rodgers,Mike McCarthy,Matt LaFleur,Brad Allen,GNB00,Lambeau Field
6297,2022_11_DAL_MIN,2022,REG,11,2022-11-20,Sunday,16:25,DAL,40.0,MIN,...,NaN,00-0033077,00-0029604,Dak Prescott,Kirk Cousins,Mike McCarthy,Kevin O'Connell,Clay Martin,MIN01,U.S. Bank Stadium


opponent

In [143]:

# grab opponent
game = schedules[
    (schedules['season'] == season) & (schedules['week'] == week) &
    ((schedules['home_team'] == team) | (schedules['away_team'] == team))]

if game['home_team'].iloc[0] == team:
        opponent = game['away_team'].iloc[0]
else:
        opponent = game['home_team'].iloc[0]

print(opponent)
# stats
opponent_weekly = weekly_stats[
            (weekly_stats['defteam'] == opponent) & 
            ((weekly_stats['season'] == season) | (weekly_stats['season'] == (season - 1)))]

opponent_weekly = opponent_weekly[~((opponent_weekly['season'] == season) & (opponent_weekly['week'] >= week))]


# points

opponent_games = schedules[
    ((schedules['home_team'] == opponent) | (schedules['away_team'] == opponent)) &
    ((schedules['season'] == season) | (schedules['season'] == (season -1)))]
opponent_games = opponent_games[~((opponent_games['season'] == season) & (opponent_games['week'] >= week))]


opponent_games

ARI


,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6147,2022_01_KC_ARI,2022,REG,1,2022-09-11,Sunday,16:25,KC,44.0,ARI,...,NaN,00-0033873,00-0035228,Patrick Mahomes,Kyler Murray,Andy Reid,Kliff Kingsbury,Scott Novak,PHO00,State Farm Stadium
6165,2022_02_ARI_LV,2022,REG,2,2022-09-18,Sunday,16:25,ARI,29.0,LV,...,NaN,00-0035228,00-0031280,Kyler Murray,Derek Carr,Kliff Kingsbury,Josh McDaniels,Clete Blakeman,VEG00,Allegiant Stadium
6180,2022_03_LA_ARI,2022,REG,3,2022-09-25,Sunday,16:25,LA,20.0,ARI,...,NaN,00-0026498,00-0035228,Matthew Stafford,Kyler Murray,Sean McVay,Kliff Kingsbury,Adrian Hill,PHO00,State Farm Stadium
6196,2022_04_ARI_CAR,2022,REG,4,2022-10-02,Sunday,16:05,ARI,26.0,CAR,...,NaN,00-0035228,00-0034855,Kyler Murray,Baker Mayfield,Kliff Kingsbury,Matt Rhule,John Hussey,CAR00,Bank of America Stadium
6213,2022_05_PHI_ARI,2022,REG,5,2022-10-09,Sunday,16:25,PHI,20.0,ARI,...,NaN,00-0036389,00-0035228,Jalen Hurts,Kyler Murray,Nick Sirianni,Kliff Kingsbury,Tra Blake,PHO00,State Farm Stadium
6227,2022_06_ARI_SEA,2022,REG,6,2022-10-16,Sunday,16:05,ARI,9.0,SEA,...,NaN,00-0035228,00-0030565,Kyler Murray,Geno Smith,Kliff Kingsbury,Pete Carroll,Shawn Hochuli,SEA00,Lumen Field
6231,2022_07_NO_ARI,2022,REG,7,2022-10-20,Thursday,20:15,NO,34.0,ARI,...,NaN,00-0027973,00-0035228,Andy Dalton,Kyler Murray,Dennis Allen,Kliff Kingsbury,Jerome Boger,PHO00,State Farm Stadium
6250,2022_08_ARI_MIN,2022,REG,8,2022-10-30,Sunday,13:00,ARI,26.0,MIN,...,NaN,00-0035228,00-0029604,Kyler Murray,Kirk Cousins,Kliff Kingsbury,Kevin O'Connell,Brad Allen,MIN01,U.S. Bank Stadium
6269,2022_09_SEA_ARI,2022,REG,9,2022-11-06,Sunday,16:05,SEA,31.0,ARI,...,NaN,00-0030565,00-0035228,Geno Smith,Kyler Murray,Pete Carroll,Kliff Kingsbury,Carl Cheffers,PHO00,State Farm Stadium
6284,2022_10_ARI_LA,2022,REG,10,2022-11-13,Sunday,16:25,ARI,27.0,LA,...,NaN,00-0027688,00-0034899,Colt McCoy,John Wolford,Kliff Kingsbury,Sean McVay,Clete Blakeman,LAX01,SoFi Stadium


## Next Edge Case: player is on a new team

### Example: Derrick Henry Wk1 2024


Gonna use new teams data from last season, rather than old teams (if week < 4)

In [204]:
# get team
player_name = 'Derrick Henry'
week = 1
season = 2024
this_week = rosters[
            (rosters['player_name'] == player_name) & 
            (rosters['season'] == season) & 
            (rosters['week'] == week)]

team = this_week['team'].iloc[0]
team

'BAL'

### Grab Position on Depth Chart?

In [ ]:
depthcharts = nfl.import_depth_charts([2022,2023,2024])

In [170]:
depthcharts[(depthcharts['formation'] == 'Offense') & (depthcharts['game_type'] == 'REG')]


,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
3,2022,ATL,1.0,REG,3,Franks,Feleipe,Feleipe,Offense,00-0036825,15,TE,FRA593646,QB,Feleipe Franks
4,2022,ATL,1.0,REG,1,Pitts,Kyle,Kyle,Offense,00-0036970,8,TE,PIT516805,TE,Kyle Pitts
5,2022,ATL,1.0,REG,1,Wilkinson,Elijah,Elijah,Offense,00-0033676,65,G,WIL099353,LG,Elijah Wilkinson
6,2022,ATL,1.0,REG,1,Lindstrom,Christopher,Chris,Offense,00-0035630,63,G,LIN451080,RG,Chris Lindstrom
7,2022,ATL,1.0,REG,1,Hesse,Parker,Parker,Offense,00-0035572,43,TE,HES209145,TE,Parker Hesse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37147,2024,HOU,19.0,REG,2,Fisher,Blake,Blake,Offense,00-0039857,57,T,FIS188380,RT,Blake Fisher
37157,2024,HOU,19.0,REG,1,Green,Kendrick,Kendrick,Offense,00-0036983,53,C,GRE298269,LG,Kendrick Green
37158,2024,HOU,19.0,REG,1,Mason,Shaquille,Shaquille,Offense,00-0031580,69,G,MAS237768,RG,Shaquille Mason
37159,2024,HOU,19.0,REG,1,Howard,Tytus,Tytus,Offense,00-0035234,71,T,HOW432152,RT,Tytus Howard


### Loop Through Rosters

In [175]:
rosters = nfl.import_weekly_rosters([2022,2023,2024])

In [193]:
player_list = rosters[((rosters['position'] == 'QB') |
                    (rosters['position'] =='RB') |
                    (rosters['position'] == 'WR') |
                    (rosters['position'] == 'TE')) &
                    (rosters['status'] == 'ACT')]

player_list = player_list[['season', 'week', 'team', 'player_name']].sort_values(['season', 'week', 'team', 'player_name'])

player_list

,season,week,team,player_name
898,2022,1,ARI,A.J. Green
24866,2022,1,ARI,Andre Baccellia
22524,2022,1,ARI,Andy Isabella
14562,2022,1,ARI,Darrel Williams
29699,2022,1,ARI,Eno Benjamin
...,...,...,...,...
121648,2024,22,PHI,Kenny Pickett
103518,2024,22,PHI,Khari Blasingame
105838,2024,22,PHI,Parris Campbell
102620,2024,22,PHI,Saquon Barkley


In [ ]:
weekly = pd.read_csv('~/Desktop/projects/Fantasy-Football-Predictor-2025/data/weekly.csv')

burrow_2021 = weekly[(weekly['player_display_name'] == 'Joe Burrow') & (weekly['season'] == 2021)]
print(burrow_2021[['week', 'passing_epa']])

      week  passing_epa
4997     1     4.165379
4998     2    -8.971513
4999     3     8.303528
5000     4    17.589502
5001     5     1.347176
5002     6    10.479851
5003     7    19.762691
5004     8    -0.316749
5005     9   -14.341417
5006    11     2.787292
5007    12     2.284419
5008    13    -5.991652
5009    14    10.851942
5010    15    -4.777006
5011    16    32.462337
5012    17    23.909231
5013    19    10.013772
5014    20    -5.045336
5015    21     2.860528
5016    22    -7.701647


In [217]:
training = pd.read_csv('~/Desktop/projects/Fantasy-Football-Predictor-2025/data/training.csv')

In [ ]:
training = training.drop(['Unnamed: 0', 'Unnamed: 0_last1'], axis = 1)